In [2]:
!pip3 install flask

In [5]:
!pip3 install twilio

     |████████████████████████████████| 460 kB 1.3 MB/s eta 0:00:01
  Created wheel for twilio: filename=twilio-6.48.0-py2.py3-none-any.whl size=1217965 sha256=009e5594f9a2a09aae472d84eebbeb4c47679ef66592a12f7dfaab27af65e6ad
  Stored in directory: /root/.cache/pip/wheels/eb/21/22/0882111898bcb77d8e8069fd61998c9515bd99fad55e506318
Successfully built twilio


In [3]:
!pip3 install flask-ngrok

In [8]:
!pip3 install numpy

In [10]:
from flask import Flask, request
from flask_ngrok import run_with_ngrok
from twilio.twiml.messaging_response import MessagingResponse
import tensorflow as tf
import tensorflow_hub as tfhub
import numpy as np

app = Flask(__name__)
run_with_ngrok(app)

model = tfhub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

perguntas = ['Como dou espaço no Python', 'espaço', 'variável em uma string','Quero aprender mais sobre Python, como faço','Uma IDE em Python']  # perguntas
respostas = ['TAB','TAB','print(f"{x}")','Acesse http://pythontutor.com/','VSCode, PyCharm']  # resposta as perguntas

tamanho = 10
embutidos = []

for i in range(0, len(perguntas), tamanho):
    embutidos.append(model(perguntas[i:i+tamanho]))
perguntas_embutidos = tf.concat(embutidos, axis=0)

def resposta(pergunta: str) -> str:
    embedding = model([pergunta,])
    # compute dot product with each question:
    probabilidade = perguntas_embutidos @ tf.transpose(embedding)

    return respostas[np.argmax(tf.squeeze(probabilidade).numpy())]

@app.route("/")
def hello():
    return "Estou no ar!"

@app.route("/wa", methods=['POST'])
def sms_reply():
    """Responder com a lógica."""
    # Fetch the message
    msg = request.form.get('Body') 
    
    # Create reply
    resp = MessagingResponse()
    resp.message("{}".format(resposta(msg)))

    return str(resp)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://11ba900a5f7b.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
